📓 Notebook 2 — 02_preprocessamento.ipynb

(Limpeza, transformação, integração, anonimização, exportação final)

In [31]:
import pandas as pd
import numpy as np


In [32]:
# ──────────────────────────────────────────
# 1. Carregar datasets
# ──────────────────────────────────────────
acidentes = pd.read_csv("../datasets/raw/acidentes_meteorologia_sp.csv")
trafego = pd.read_csv("../datasets/raw/mobilidade_urbana_trafego.csv")
transporte = pd.read_csv("../datasets/raw/transporte_publico_politicas_alinhado.csv")

In [33]:
# ──────────────────────────────────────────
# 2. Normalização dos nomes das colunas
# ──────────────────────────────────────────
def normalizar(df):
    df.columns = df.columns.str.strip().str.lower()
    return df

trafego = normalizar(trafego)
acidentes = normalizar(acidentes)
transporte = normalizar(transporte)

In [34]:
# ──────────────────────────────────────────
# 3. Tratamento de datas e horas
# ──────────────────────────────────────────
for df in (acidentes, trafego, transporte):
    if "data" in df:
        df["data"] = pd.to_datetime(df["data"], errors="coerce")

    if "hora" in df:
        df["hora"] = pd.to_datetime(df["hora"], format="%H:%M", errors="ignore")\
                        .apply(lambda x: x.hour if hasattr(x, "hour") else np.nan)

# Criar colunas temporais
for df in (acidentes, trafego, transporte):
    if "data" in df:
        df["ano"] = df["data"].dt.year
        df["mes"] = df["data"].dt.month
        df["dia_semana"] = df["data"].dt.day_name(locale="pt")

C:\Users\sem nome\AppData\Local\Temp\ipykernel_16788\1737526069.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df["hora"] = pd.to_datetime(df["hora"], format="%H:%M", errors="ignore")\
C:\Users\sem nome\AppData\Local\Temp\ipykernel_16788\1737526069.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df["hora"] = pd.to_datetime(df["hora"], format="%H:%M", errors="ignore")\


In [35]:
# ──────────────────────────────────────────
# 4. Padronização de strings (via, cidade, bairro)
# ──────────────────────────────────────────
for df in (acidentes, trafego, transporte):
    for col in ["via", "cidade", "bairro"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.upper().str.strip()

In [36]:
# ──────────────────────────────────────────
# 5. Tratamento de valores ausentes
# ──────────────────────────────────────────
trafego["volume_veiculos"] = trafego["volume_veiculos"].fillna(trafego["volume_veiculos"].median())
trafego["velocidade_media"] = trafego["velocidade_media"].fillna(trafego["velocidade_media"].median())

acidentes["numero_feridos"] = acidentes["numero_feridos"].fillna(0)
acidentes["numero_mortos"] = acidentes["numero_mortos"].fillna(0)

transporte["n_passageiros"] = transporte["n_passageiros"].fillna(0)

In [37]:
# ──────────────────────────────────────────
# 6. Integração dos 3 datasets
# ──────────────────────────────────────────
base = acidentes.merge(
    trafego,
    on=["data", "hora", "via"],
    how="outer",
    suffixes=("_acidentes", "_trafego")
)

dados = base.merge(
    transporte,
    on=["data"],
    how="outer"
)

In [38]:
# ──────────────────────────────────────────
# 7. Anonimização (coordenadas aproximadas)
# ──────────────────────────────────────────
if "latitude" in dados:
    dados["latitude"] = dados["latitude"].round(3)
if "longitude" in dados:
    dados["longitude"] = dados["longitude"].round(3)

In [39]:
# ──────────────────────────────────────────
# 8. Seleção final de colunas úteis
# ──────────────────────────────────────────
colunas = [
    "data","hora","dia_semana","volume_veiculos","velocidade_media","via","bairro","nivel_congestionamento","clima","tipo_acidente","gravidade",
    "temperatura","numero_feridos","numero_mortos",
    "n_passageiros",
    "tipo_transporte","capacidade",
    "politica","variacao_trafego",
    "variacao_acidentes","vento","humidade","categoria_clima",
    "ano","mes"
]

dados = dados[[c for c in colunas if c in dados.columns]]

In [40]:
# ──────────────────────────────────────────
# 9. Salvar dataset final
# ──────────────────────────────────────────
dados.to_csv("../datasets/processed/dados_mobilidade_integrados.csv", index=False)

print("Pré-processamento concluído e dataset exportado.")

Pré-processamento concluído e dataset exportado.
